ITMO Computer Vision Course 2022

---

# Лабораторная работа №3

**Простые системы классификации изображений на основе сверточных нейронных сетей**



---

Перед запуском:


*   Нужно подгрузить лейблы классов - см. закомментированные строки в коде
*   Датасет для определения нужно положить в папку /samples/, которую нужно создать - тоже смотри код

Вариант 1 - AlexNet:

In [35]:
#Подгружаем данные
# Download ImageNet labels
!wget https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt

#Загружаем 
!wget https://db.rzhevskyrobotics.com/s/aszMPeLN2Y4CcQS/download/samples.zip
!unzip samples.zip

#Система автоматом создает эту ерунду, а библа пытается там прочитать фотки
!rm -R ./samples/.ipynb_checkpoints

--2023-03-07 19:32:03--  https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10472 (10K) [text/plain]
Saving to: ‘imagenet_classes.txt’

imagenet_classes.tx 100%[===================>]  10.23K  --.-KB/s    in 0s      

2023-03-07 19:32:03 (54.1 MB/s) - ‘imagenet_classes.txt’ saved [10472/10472]

--2023-03-07 19:32:04--  https://db.rzhevskyrobotics.com/s/aszMPeLN2Y4CcQS/download/samples.zip
Resolving db.rzhevskyrobotics.com (db.rzhevskyrobotics.com)... 91.227.47.147
Connecting to db.rzhevskyrobotics.com (db.rzhevskyrobotics.com)|91.227.47.147|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 25304867 (24M) [application/zip]
Saving to: ‘samples.zip’

samples.zip       

In [39]:
#
#Основной скрипт
#
from torchvision import transforms
from torchvision import models
import torch
from PIL import Image

import os
import psutil

import time
from os import walk

#
# Функции для анализа потребляемой памяти
#

# inner psutil function
def process_memory():
    process = psutil.Process(os.getpid())
    mem_info = process.memory_info()
    return mem_info.rss
 
# decorator function
def profile(func):
    def wrapper(*args, **kwargs):
 
        mem_before = process_memory()
        result = func(*args, **kwargs)
        mem_after = process_memory()
        print("{}:consumed memory: {:,}".format(
            func.__name__,
            mem_before, mem_after, mem_after - mem_before))
 
        return result
    return wrapper


#
# Функция для получения самых повторяемых меток
#
def sortList(labels_list):
  li = sorted(set(labels_list), key = lambda ele: labels_list.count(ele))
  return list(reversed(li))[:5]


#
# Основной процесс
#

@profile
def main_process(input_batch):
  alexnet = models.alexnet(pretrained=True)

  alexnet.eval()

  # Читаем файл с категориями. По идее, 1000 классов
  with open("imagenet_classes.txt", "r") as f:
      categories = [s.strip() for s in f.readlines()]

  #Время исполнения начало
  start = time.time()

  #Считываем изображения
  current_item  = 1
  labels_list = []
  for current_data in input_batch:

    print(">>>>> Item {} :".format(current_item))

    # Если можем, то заливаем на GPU
    if torch.cuda.is_available():
        current_data = current_data.to('cuda')
        alexnet.to('cuda')

    with torch.no_grad():
        output = alexnet(current_data)

    # Нормализуем скоры через софтмакс для получения вероятностей. 
    probabilities = torch.nn.functional.softmax(output[0], dim=0)
    #print(probabilities)


    # Для каждого изображения показываем топ-5
    top5_prob, top5_catid = torch.topk(probabilities, 5)
    for i in range(top5_prob.size(0)):
        labels_list.append(categories[top5_catid[i]])
        print(categories[top5_catid[i]], top5_prob[i].item())

    current_item = current_item +1


  #Время исполнения конец
  end = time.time()
  #Печатаем время исполнения
  print("Time consuption: {} s".format((end-start)))

  print(sortList(labels_list))


#
# НАЧАЛО СКРИПТА
#

input_batch = []

#Пока без проверки :)
filenames = next(walk("./samples"), (None, None, []))[2]

#Загружаем изображения, нормируем их, загружаем в микробатчик
for image_name in filenames:
  input_image = Image.open("./samples/"+image_name)
  preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
  ])
  input_tensor = preprocess(input_image)  
  current_batch = input_tensor.unsqueeze(0)
  input_batch.append(current_batch)

main_process(input_batch)


/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


>>>>> Item 1 :
red fox 0.8774667978286743
kit fox 0.1100539043545723
papillon 0.003329595783725381
Pembroke 0.0022576875053346157
dhole 0.0013380349846556783
>>>>> Item 2 :
red fox 0.9813829064369202
grey fox 0.010669006034731865
kit fox 0.005031710024923086
red wolf 0.0014204097678884864
dhole 0.0003951021935790777
>>>>> Item 3 :
red fox 0.9561010599136353
kit fox 0.02248392626643181
weasel 0.00955805554986
Arctic fox 0.0029835703317075968
grey fox 0.0014872472966089845
>>>>> Item 4 :
red fox 0.7397969365119934
kit fox 0.2466931790113449
grey fox 0.004727737512439489
dhole 0.004473443608731031
red wolf 0.002235488034784794
>>>>> Item 5 :
red fox 0.9029809236526489
kit fox 0.09444959461688995
grey fox 0.0022012502886354923
red wolf 9.835192031459883e-05
Pembroke 6.484986806754023e-05
>>>>> Item 6 :
red fox 0.5401155352592468
kit fox 0.2568546235561371
Pembroke 0.12628506124019623
red wolf 0.017396260052919388
timber wolf 0.014975818805396557
>>>>> Item 7 :
red fox 0.47031500935554504
k

Теперь ResNet101, тоже тренировали на ImageNet
https://pytorch.org/hub/pytorch_vision_resnet/ 

In [40]:
#
#Основной скрипт
#
from torchvision import transforms
from torchvision import models
import torch
from PIL import Image

import os
import psutil

import time
from os import walk

#
# Функции для анализа потребляемой памяти
#

# inner psutil function
def process_memory():
    process = psutil.Process(os.getpid())
    mem_info = process.memory_info()
    return mem_info.rss
 
# decorator function
def profile(func):
    def wrapper(*args, **kwargs):
 
        mem_before = process_memory()
        result = func(*args, **kwargs)
        mem_after = process_memory()
        print("{}:consumed memory: {:,}".format(
            func.__name__,
            mem_before, mem_after, mem_after - mem_before))
 
        return result
    return wrapper


#
# Функция для получения самых повторяемых меток
#
def sortList(labels_list):
  li = sorted(set(labels_list), key = lambda ele: labels_list.count(ele))
  return list(reversed(li))[:5]


#
# Основной процесс
#

@profile
def main_process(input_batch):
  resnet = models.resnet101(pretrained=True)

  resnet.eval()

  # Читаем файл с категориями. По идее, 1000 классов
  with open("imagenet_classes.txt", "r") as f:
      categories = [s.strip() for s in f.readlines()]

  #Время исполнения начало
  start = time.time()

  #Считываем изображения
  current_item  = 1
  labels_list = []
  for current_data in input_batch:

    print(">>>>> Item {} :".format(current_item))

    # Если можем, то заливаем на GPU
    if torch.cuda.is_available():
        current_data = current_data.to('cuda')
        resnet.to('cuda')

    with torch.no_grad():
        output = resnet(current_data)

    # Нормализуем скоры через софтмакс для получения вероятностей. 
    probabilities = torch.nn.functional.softmax(output[0], dim=0)
    #print(probabilities)

    # Для каждого изображения показываем топ-5
    top5_prob, top5_catid = torch.topk(probabilities, 5)
    for i in range(top5_prob.size(0)):
        labels_list.append(categories[top5_catid[i]])
        print(categories[top5_catid[i]], top5_prob[i].item())

    current_item = current_item +1


  #Время исполнения конец
  end = time.time()
  #Печатаем время исполнения
  print("Time consuption: {} s".format((end-start)))

  print(sortList(labels_list))


#
# НАЧАЛО СКРИПТА
#

input_batch = []

#Пока без проверки :)
filenames = next(walk("./samples"), (None, None, []))[2]

#Загружаем изображения, нормируем их, загружаем в микробатчик
for image_name in filenames:
  input_image = Image.open("./samples/"+image_name)
  preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
  ])
  input_tensor = preprocess(input_image)  
  current_batch = input_tensor.unsqueeze(0)
  input_batch.append(current_batch)

main_process(input_batch)

>>>>> Item 1 :
red fox 0.9961419701576233
kit fox 0.0034042620100080967
grey fox 0.00031536640017293394
dhole 9.240717918146402e-05
Arctic fox 1.672488906478975e-05
>>>>> Item 2 :
red fox 0.98582524061203
kit fox 0.007947615347802639
grey fox 0.005385344382375479
Arctic fox 0.0005776234320364892
coyote 0.00010337311687180772
>>>>> Item 3 :
red fox 0.9066956043243408
grey fox 0.05715973675251007
kit fox 0.034011248499155045
Arctic fox 0.0011702527990564704
dhole 0.0003072912513744086
>>>>> Item 4 :
red fox 0.9366707801818848
kit fox 0.0436943955719471
grey fox 0.018678031861782074
Arctic fox 0.0002649485832080245
dhole 0.00020816968753933907
>>>>> Item 5 :
red fox 0.9625715613365173
kit fox 0.03354345262050629
grey fox 0.0035065629053860903
red wolf 0.0001630091283004731
dhole 8.570853970013559e-05
>>>>> Item 6 :
red fox 0.958617627620697
grey fox 0.017909692600369453
kit fox 0.015125230886042118
red wolf 0.003647872246801853
Pembroke 0.0009068035869859159
>>>>> Item 7 :
kit fox 0.67355

Теперь пробуем VGG16:

In [41]:
#
#Основной скрипт
#
from torchvision import transforms
from torchvision import models
import torch
from PIL import Image

import os
import psutil

import time
from os import walk

#
# Функции для анализа потребляемой памяти
#

# inner psutil function
def process_memory():
    process = psutil.Process(os.getpid())
    mem_info = process.memory_info()
    return mem_info.rss
 
# decorator function
def profile(func):
    def wrapper(*args, **kwargs):
 
        mem_before = process_memory()
        result = func(*args, **kwargs)
        mem_after = process_memory()
        print("{}:consumed memory: {:,}".format(
            func.__name__,
            mem_before, mem_after, mem_after - mem_before))
 
        return result
    return wrapper


#
# Функция для получения самых повторяемых меток
#
def sortList(labels_list):
  li = sorted(set(labels_list), key = lambda ele: labels_list.count(ele))
  return list(reversed(li))[:5]


#
# Основной процесс
#

@profile
def main_process(input_batch):
  vgg16 = models.vgg16(pretrained=True)

  vgg16.eval()

  # Читаем файл с категориями. По идее, 1000 классов
  with open("imagenet_classes.txt", "r") as f:
      categories = [s.strip() for s in f.readlines()]

  #Время исполнения начало
  start = time.time()

  #Считываем изображения
  current_item  = 1
  labels_list = []
  for current_data in input_batch:

    print(">>>>> Item {} :".format(current_item))

    # Если можем, то заливаем на GPU
    if torch.cuda.is_available():
        current_data = current_data.to('cuda')
        vgg16.to('cuda')

    with torch.no_grad():
        output = vgg16(current_data)

    # Нормализуем скоры через софтмакс для получения вероятностей. 
    probabilities = torch.nn.functional.softmax(output[0], dim=0)
    #print(probabilities)

    #Время исполнения конец
    end = time.time()

    # Для каждого изображения показываем топ-5
    top5_prob, top5_catid = torch.topk(probabilities, 5)
    for i in range(top5_prob.size(0)):
        labels_list.append(categories[top5_catid[i]])
        print(categories[top5_catid[i]], top5_prob[i].item())

    current_item = current_item +1


  #Время исполнения конец
  end = time.time()
  #Печатаем время исполнения
  print("Time consuption: {} s".format((end-start)))

  print(sortList(labels_list))


#
# НАЧАЛО СКРИПТА
#

input_batch = []

#Пока без проверки :)
filenames = next(walk("./samples"), (None, None, []))[2]

#Загружаем изображения, нормируем их, загружаем в микробатчик
for image_name in filenames:
  input_image = Image.open("./samples/"+image_name)
  preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
  ])
  input_tensor = preprocess(input_image)  
  current_batch = input_tensor.unsqueeze(0)
  input_batch.append(current_batch)

main_process(input_batch)

>>>>> Item 1 :
red fox 0.964917004108429
kit fox 0.03098888322710991
grey fox 0.0026827016845345497
Pembroke 0.0008605083567090333
Arctic fox 0.0001531234011054039
>>>>> Item 2 :
red fox 0.9720110297203064
kit fox 0.023280955851078033
grey fox 0.004540382418781519
dhole 7.693743100389838e-05
red wolf 4.229386104270816e-05
>>>>> Item 3 :
red fox 0.9375585317611694
kit fox 0.04601653665304184
grey fox 0.015913326293230057
coyote 0.0002569016651250422
red wolf 9.037856216309592e-05
>>>>> Item 4 :
red fox 0.9180614948272705
kit fox 0.07899190485477448
grey fox 0.0026812138967216015
Arctic fox 0.00011230208474444225
coyote 8.304709626827389e-05
>>>>> Item 5 :
red fox 0.9537447094917297
kit fox 0.040751297026872635
grey fox 0.0037162858061492443
coyote 0.0006098044686950743
dhole 0.0005456835497170687
>>>>> Item 6 :
red fox 0.9597309231758118
kit fox 0.029443368315696716
grey fox 0.003594473237171769
Shetland sheepdog 0.0017681290628388524
Pembroke 0.0017552061472088099
>>>>> Item 7 :
red fo